In [1]:
import json
from ast import literal_eval
from easynmt import EasyNMT
from tqdm import tqdm_notebook

In [2]:
with open("EgeEvalDataset.json", 'r', encoding="utf8") as inp:
    dataEGE = json.load(inp)

In [3]:
for index, item in enumerate(dataEGE):
    dataEGE[index]["distractors"] = literal_eval(item["distractors"])

In [4]:
with open("EgeEvalDataset.json", 'w', encoding="utf8") as outp:
    json.dump(dataEGE, outp, indent=2, ensure_ascii=False)

In [5]:
LB = '\n'

In [6]:
data_ege_to_translate = [
    f"{item['reading_text']}{LB}{item['question']}{LB}{item['right_answer']}{LB}{LB.join([distractor for distractor in item['distractors']])}"
    for item in dataEGE
]

In [8]:
model = EasyNMT('opus-mt')

11.9kB [00:00, 14.4MB/s]                   


In [10]:
print(model.translate(
    data_ege_to_translate[0],
    source_lang='ru', target_lang='en'
))

 On the back of our village, there was a long, boarded room. This is the first time I've heard music in my life, the violin. She was played by Vasya Poliac. What did the music tell me? Something very big. What did she complain about, who she was angry with? Worried and bitter to me. I want to cry because I feel sorry for myself, and I feel sorry for those who sleep in the graveyard! Wasa continued to play, saying: "This music was written by a man who had been deprived of the most precious one. If a man has no mother, no father, but a homeland, he's not an orphan yet. Everything goes by: love, regret, grief, even pain from wounds — but it never goes away, and it doesn't go away. This music was written by my countryman Oginsky. He wrote at the border saying goodbye to his motherland. He sent her last hello. There has been no composer in the world for a long time, but his pain, his sorrow, love for his own land, which no one can take away, is still alive." "Thank you, Uncle," I whispered.

In [13]:
translated_ege_data = []

In [15]:
for item in tqdm_notebook(
    data_ege_to_translate, total=len(data_ege_to_translate)
):
    translated_item = model.translate(
        item, source_lang="ru", target_lang="en"
    )
    translated_ege_data.append(translated_item)

/tmp/ipykernel_13494/2438184859.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for item in tqdm_notebook(


  0%|          | 0/55 [00:00<?, ?it/s]

In [16]:
len(translated_ege_data)

55

In [23]:
translated_ege_data_structured = []

In [24]:
for item in translated_ege_data:
    item_structured = item.split('\n')
    text = item_structured[0]
    question = item_structured[1]
    right_answer = item_structured[2]
    distractors = item_structured[3:]
    new_item = {
        "reading_text": text,
        "question": question,
        "right_answer": right_answer,
        "distractors": distractors
    }
    translated_ege_data_structured.append(new_item)

In [25]:
with open("EgeEvalDataset_translated.json", 'w', encoding='utf8') as outp:
    json.dump(
        translated_ege_data_structured, outp,
        indent=4, ensure_ascii=False
    )